# The Plan
Everything we know about a specific plan. 

### Instructions
1. Run all cells! (click on Menu > Cell > Run All Cells)
1. View report at the bottom.

In [ ]:
planId = "e379b1c6-ffe1-4d1f-9c85-841e15a87631"

In [ ]:
%%capture 

# install packages, setup workspace root
!pip install --upgrade azure-kusto-notebooks
import os
import sys
import datetime
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import concurrent.futures
from azure.kusto.notebooks import utils as akn

# cwd should be workspace root
if os.path.basename(os.getcwd()) == 'devops-pipelines':
    os.chdir(os.pardir)

In [ ]:
# authenticate kusto client
# you will need to copy the token into a browser window for AAD auth. 
client = akn.get_client('https://vso.kusto.windows.net', 'VSO')

In [ ]:
# Load basic plan info
query = os.path.join('devops-pipelines', 'queries', 'run', 'PlanInfo.csl')
response = akn.execute_file(client, 'VSO', query, {'OrchestrationId': akn.quote(planId)})
result = response.primary_results[0]

In [ ]:
import importlib
importlib.reload(akn)

r = akn.Report()
r.write('# Basic Info')
r.write(akn.row_as_markdown_table(akn.to_dataframe(result)))
from IPython.display import Markdown
Markdown(r.content)

In [ ]:
# Load WhatHappened
query = os.path.join('devops-pipelines', 'queries', 'run', 'WhatHappened.csl')
result = akn.execute_file(client, 'VSO', query, {'OrchestrationId': akn.quote(planId)})
history = akn.to_dataframe(result.primary_results[0])

In [ ]:
# compute relative time stamps
t0 = history['PreciseTimeStamp'].iloc[0]
history['Time'] = history.apply(lambda row: row['PreciseTimeStamp'] - t0, axis=1)
history.OrchestrationId = history.apply(lambda row: row.OrchestrationId[37:], axis=1)

# record critical times
def find_time(message):
    r = history[history.Message.str.startswith(message)]
    if len(r.index) > 0:
        return r['PreciseTimeStamp'].iloc[0]

create_time = find_time('Created plan')
start_time = find_time('Started plan')
end_time = find_time('Completed orchestration with result')

In [ ]:
columns_to_ignore = ('source_', 'PreciseTimeStamp')
columns = ['Time'] + [c for c in history.columns if c not in columns_to_ignore and c != 'Time']
pd.set_option('max_colwidth', 200)


from IPython.display import HTML
HTML(history[columns].to_html(index=False, notebook=True))
# Markdown(akn.pandas_df_to_markdown_table(history[columns]))